In [16]:
import os
import pandas as pd

In [20]:
dir_path = "benchmarks"
file_list = os.listdir(dir_path)

df_list = []
for filename in file_list:
    df = pd.read_csv(f"{dir_path}/{filename}", nrows=16, header=1, sep="\\s+")
    df["filename"] = filename
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)

df

,#type,avg,std,min,p5,p10,p50,p67,p75,p80,p85,p90,p95,p99,p999,p9999,QPS,target,filename
0,read,261.2,426.2,74.4,132.3,144.1,185.6,205.3,218.6,230.8,247.4,277.1,674.5,1523.3,7672.7,9165.6,4932.6,5000,some-memcached_run-2.txt
1,read,307.2,424.8,74.4,127.9,149.2,215.8,246.0,264.5,279.0,301.9,364.0,1201.8,2175.5,7217.3,8104.5,9914.2,10000,some-memcached_run-2.txt
2,read,362.1,479.1,68.9,118.7,142.1,241.6,279.4,303.6,326.2,374.3,517.6,1340.8,2249.5,7341.7,8122.0,15066.2,15000,some-memcached_run-2.txt
3,read,383.7,471.3,74.4,118.9,139.0,257.8,301.5,332.2,363.9,432.7,819.5,1359.2,2237.3,7095.2,8110.4,20004.7,20000,some-memcached_run-2.txt
4,read,407.2,466.2,68.9,117.4,137.8,276.3,326.6,364.3,405.5,503.5,1063.6,1372.9,2035.4,6102.1,8077.2,25100.2,25000,some-memcached_run-2.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,read,791.7,494.9,68.9,191.8,263.0,819.2,940.7,993.2,1029.9,1080.9,1188.4,1586.7,2012.7,7279.2,8470.6,59892.4,60000,ibench-l1d_run-3.txt
332,read,1119.8,524.2,86.8,791.7,847.3,1026.6,1084.8,1124.1,1158.6,1205.2,1454.2,1946.0,2274.8,7897.7,9657.4,59484.5,65000,ibench-l1d_run-3.txt
333,read,1122.8,517.6,80.4,796.3,849.5,1026.5,1086.5,1130.1,1166.7,1230.7,1466.6,1939.4,2290.4,7873.1,9616.0,59550.8,70000,ibench-l1d_run-3.txt
334,read,1113.8,512.7,148.8,810.0,854.8,1019.1,1075.0,1102.5,1142.2,1182.0,1390.8,1933.7,2221.8,7862.7,11907.6,60116.2,75000,ibench-l1d_run-3.txt
